In [1]:
from __future__ import print_function, division
from pysis.util import file_variations
import os
from pysis import CubeFile
from pysis.isis import cubenorm, handmos, hi2isis, hical, histitch, spiceinit, getkey
import subprocess as sp
from pysis.exceptions import ProcessError
import sys

# case 2: turn IMG to cub, give information of it (spiceinit), don't calibrate it
def nocal_hi(img_name):
    (img_name, cub_name) = file_variations(img_name, ['.IMG', '.cub'])
    # something wrong here, if just put one extension in it, the name will contain brankets
    try:
        hi2isis(from_=img_name, to=cub_name)
        spiceinit(from_=cub_name)
    except ProcessError as e:
        print(e.stdout)
        print(e.stderr)
        sys.exit()

# stitch _0 and _1 together and normalize it
def stit_norm_hi(img1, img2):
    cub = img1[0:20]+'.cub'
    (cub, norm) = file_variations(cub, ['.cub', '.norm.cub'])   # same here
    try:
        histitch(from1=img1, from2=img2, to=cub)
        cubenorm(from_=cub, to=norm)
    except ProcessError as e:
        print(e.stdout)
        print(e.stderr)
        sys.exit()
        
def hi2mos(nm):
    os.chdir('/Users/klay6683/data/planet4/season2_3_EDRs')
    print('Processing the EDR data associated with '+nm)

    mos_name = 'redMosaic'+nm+'.cub'
#     status = os.path.isfile(mos_name)
    status = False
    if status is True:
        print('skip processing '+nm+'...')
        return nm, False
    else:
        nm = nm+'_RED'
        channel = [4, 5]
        ccd = [0, 1]

        for c in channel:
            for chip in ccd:
                nocal_hi(nm+str(c)+'_'+str(chip)+'.IMG')

            stit_norm_hi(nm+str(c)+'_0.cub', nm+str(c)+'_1.cub')

        # handmos part
        im0 = CubeFile.open(nm+'4.norm.cub')  # use CubeFile to get lines and samples
        # use linux commands to get binning mode
        bin = int(getkey(from_=nm+'4.norm.cub', objname="isiscube", grpname="instrument",
                  keyword="summing"))

        # because there is a gap btw RED4 & 5, nsamples need to first make space
        # for 2 cubs then cut some overlap pixels
        try:
            handmos(from_=nm+'4.norm.cub', mosaic=mos_name, nbands=1, outline=1, outband=1,
                    create='Y', outsample=1, nsamples=im0.samples*2-48//bin, 
                    nlines=im0.lines)
        except ProcessError as e:
            print("STDOUT:", e.stdout)
            print("STDERR:", e.stderr)
        im0 = CubeFile.open(nm+'5.norm.cub')  # use CubeFile to get lines and samples

        # deal with the overlap gap between RED4 & 5:
        handmos(from_=nm+'5.norm.cub', mosaic=mos_name, outline=1, outband=1, create='N',
                outsample=im0.samples-48//bin+1)  
        return nm, True    


def cleanup(data_dir, img):
    # do some cleanup removing temporary files
    # removing ISIS cubes made during processing that aren't needed
    fs = glob.glob(data_dir+'/'+img+'_RED*.cub')

    print(fs)
    for f in fs:
        os.remove(f)

    # removing the normalized files

    fs = glob.glob(data_dir+'/'+img+'_RED*.norm.cub')
    for f in fs:
        os.remove(f)

    # remove the raw EDR data

    fs = glob.glob(data_dir+'/'+img+'_RED*.IMG')
    for f in fs:
        os.remove(f)



In [3]:
img_names = pd.read_table('/Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt',
                          header=None, squeeze=True)
img_names.head()

0    ESP_013113_0985
1    ESP_013034_0985
2    ESP_012889_0985
3    ESP_012744_0985
4    ESP_012691_0985
Name: 0, dtype: object

In [5]:
cd ~/data/planet4/season2_3_EDRs/

/Users/klay6683/data/planet4/season2_3_EDRs


In [107]:
from ipyparallel import Client
c = Client()

In [108]:
lbview = c.load_balanced_view()
dview = c.direct_view()

In [8]:
%%px 
from __future__ import print_function,division
from pysis.util import file_variations
import os
from pysis import CubeFile
from pysis.isis import cubenorm, handmos, hi2isis, hical, histitch, spiceinit, getkey
import subprocess as sp
from pysis.exceptions import ProcessError
import sys

In [9]:
dview.push({'nocal_hi':nocal_hi,
            'stit_norm_hi':stit_norm_hi})

<AsyncResult: finished>

In [12]:
results = lbview.map_async(hi2mos, img_names)

In [8]:
from iuvs.multitools import nb_progress_display

In [14]:
nb_progress_display(results, img_names)

In [15]:
for res in results:
    print(res)

('ESP_013113_0985_RED', True)
('ESP_013034_0985_RED', True)
('ESP_012889_0985_RED', True)
('ESP_012744_0985_RED', True)
('ESP_012691_0985_RED', True)
('ESP_012467_0985_RED', True)
('ESP_012322_0985_RED', True)
('ESP_012256_0985_RED', True)
('ESP_011900_0985_RED', True)
('ESP_011729_0985_RED', True)
('ESP_011702_0985_RED', True)
('ESP_011623_0985_RED', True)
('ESP_011557_0985_RED', True)
('ESP_011544_0985_RED', True)
('ESP_011491_0985_RED', True)
('ESP_022699_0985_RED', True)
('ESP_021460_0985_RED', True)
('ESP_020959_0985_RED', True)
('ESP_020748_0985_RED', True)
('ESP_020194_0985_RED', True)
('ESP_020128_0985_RED', True)
('ESP_020049_0985_RED', True)
('ESP_021829_0985_RED', True)
('ESP_021684_0985_RED', True)
('ESP_021671_0985_RED', True)
('ESP_021605_0985_RED', True)
('ESP_021526_0985_RED', True)
('ESP_020827_0985_RED', True)
('ESP_020339_0985_RED', True)
('ESP_020115_0985_RED', True)


## cleanup

In [17]:
!rm -f *RED*.cub

# xy2latlon

In [10]:
df = pd.read_hdf('/Users/klay6683/Dropbox/data/planet4/seasons2and3_v2/ESP_011296_0975_blotches.hdf')
df.head()

,x,y,angle,radius_1,radius_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,n_members
0,696.900000,7205.600000,40.524806,28.788229,19.301067,718.782644,7214.543650,675.017356,7196.656350,684.358607,7220.271218,709.441393,7190.928782,5
1,504.371528,6903.006945,-6.437134,13.623201,12.144633,517.908841,6900.917949,490.834215,6905.095941,505.733099,6915.075011,503.009957,6890.938878,24
2,385.189815,6753.222222,-13.839050,24.177703,18.880157,408.665675,6730.110967,361.713955,6776.333477,389.705860,6771.554316,380.673771,6734.890128,18
3,779.250000,7551.300000,3.560808,11.718488,11.278663,790.945865,7546.530061,767.554135,7556.069939,778.549507,7562.556889,779.950493,7540.043111,10
4,833.062500,4602.000000,-9.765882,14.234663,12.238497,847.090889,4606.761891,819.034111,4597.238109,835.138427,4614.061150,830.986573,4589.938850,8


In [11]:
from pysis.isis import campt
from pysis.exceptions import ProcessError
import pvl

In [18]:
from pathlib import Path
edrpath = Path('/Users/klay6683/data/planet4/season2_3_EDRs/')
clusterpath = Path('/Users/klay6683/Dropbox/data/planet4/seasons2and3_v2/')

In [19]:
obsids = !cat /Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt

In [57]:
fpaths = [item for obsid in obsids for item in (clusterpath.glob("{}_*.hdf".format(obsid)))]

In [140]:
blotch_coords = ['', 'p1', 'p2', 'p3', 'p4']
fan_coords = ['', 'arm1', 'arm2']
fnotch_coords = ['blotch_'+c for c in blotch_coords]
fnotch_coords += ['fan_'+c for c in fan_coords]

In [141]:
fnotch_coords

['blotch_',
 'blotch_p1',
 'blotch_p2',
 'blotch_p3',
 'blotch_p4',
 'fan_',
 'fan_arm1',
 'fan_arm2']

In [166]:
with dview.sync_imports():
    from pysis.isis import campt
    from pysis.exceptions import ProcessError
    from pathlib import Path
    from ipyparallel import CompositeError

importing campt from pysis.isis on engine(s)
importing ProcessError from pysis.exceptions on engine(s)
importing Path from pathlib on engine(s)
importing CompositeError from ipyparallel on engine(s)


In [185]:
def do_campt(mosaicname, savepath, temppath):
    try:
        campt(from_=mosaicname, to=savepath, format='flat', append='no',
              coordlist=temppath, coordtype='image')
    except ProcessError as e:
        print(e.stderr)
        return obsid, False


def process_inpath(inpath, marking, mosaicpath):
    coords_switch = dict(blotches=blotch_coords,
                         fans=fan_coords,
                         fnotches=fnotch_coords)
        
    df = pd.read_hdf(str(inpath), 'df')
    for coord in coords_switch[marking]:
        if coord.endswith('_'):
            tempcoords = [coord + 'x', coord + 'y']
        elif coord == '':
            tempcoords = ['x', 'y']
        else:
            tempcoords = [coord + '_x', coord + '_y']
        if marking == 'fnotches':
            tempcoords
        temppath = inpath.with_suffix('.tocampt')
        try:
            df[tempcoords].to_csv(str(temppath), header=False, index=False)
        except KeyError:
            return False
        name = coord
        if name == '' or name.endswith('_'): 
            name += 'base'
        savename = "{stem}_{c}_campt_out.csv".format(stem=inpath.stem, c=name)
        savepath = clusterpath / savename
#         do_campt(mosaicpath, savepath, temppath)
        return True

def xy2latlon(inpath):
    d = dict(inpath=inpath)
    edrpath = Path('/Users/klay6683/data/planet4/season2_3_EDRs/')
    tokens = inpath.stem.split('_')
    obsid = '_'.join(tokens[:3])
    marking = tokens[-1]
    mosaicname = 'redMosaic' + obsid + '.cub'
    mosaicpath = edrpath / mosaicname
    ok = process_inpath(inpath, marking, mosaicpath)
    d['ok'] = ok
    return d

In [197]:
dview.push(dict(process_inpath=process_inpath,
                do_campt=do_campt,
                blotch_coords=blotch_coords,
                fan_coords=fan_coords,
                fnotch_coords=fnotch_coords,
                edrpath=Path('/Users/klay6683/data/planet4/season2_3_EDRs/'),
                clusterpath=Path('/Users/klay6683/Dropbox/data/planet4/seasons2and3_v2/')))

<AsyncResult: finished>

In [169]:
xy2latlon(fpaths[2])

('ESP_013113_0985', True)

In [198]:
results = lbview.map_async(xy2latlon, fpaths)

In [203]:
res = pd.DataFrame(results.result)

In [206]:
res[res.ok==False].inpath.values

array([ PosixPath('/Users/klay6683/Dropbox/data/planet4/seasons2and3_v2/ESP_022699_0985_fnotches.hdf')], dtype=object)

In [188]:
from iuvs.multitools import nb_progress_display

In [195]:
nb_progress_display(results, obsids)